ref.
- https://www.youtube.com/watch?v=3G3zExNItj0&t=303&ab_channel=%EC%A1%B0%EC%BD%94%EB%94%A9JoCoding
- https://www.youtube.com/watch?v=13wirJFYAvQ&ab_channel=nolGaeMiTV 

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
import seaborn as sns
import tqdm

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier            # Voting 기법 사용 Module
from sklearn.neighbors import KNeighborsClassifier 
import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

Data Load

In [2]:
df = pd.read_csv('./data/lotto.csv', index_col=0)
df

,1,2,3,4,5,6,bonus,1st,2nd,3rd,4th,5th
num,,,,,,,,,,,,
1,10,23,29,33,37,40,16,0,143934100,5140500,113400,10000
2,9,13,21,25,32,42,2,2002006800,94866800,1842000,100800,10000
3,11,16,19,21,27,31,30,2000000000,0,1174100,54900,10000
4,14,27,30,31,40,42,2,0,211191200,7282400,152100,10000
5,16,24,29,40,41,42,3,0,0,6033800,166500,10000
...,...,...,...,...,...,...,...,...,...,...,...,...
1082,21,26,27,32,34,42,31,3720489643,70009214,1568126,50000,5000
1083,3,7,14,15,22,38,17,1713084525,59482102,1308098,50000,5000
1084,8,12,13,29,33,42,5,1738764600,40625342,1447523,50000,5000


In [3]:
print(df.index)
print(df.columns)

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086],
           dtype='int64', name='num', length=1086)
Index(['1', '2', '3', '4', '5', '6', 'bonus', '1st', '2nd', '3rd', '4th',
       '5th'],
      dtype='object')


One Hot Encoding

In [4]:
# one-hot encoding

# 당첨번호를 원핫인코딩벡터(ohbin)으로 변환
def numbers2ohbin(numbers):

    ohbin = np.zeros(45) #45개의 빈 칸을 만듬

    for i in range(6): #여섯개의 당첨번호에 대해서 반복함
        ohbin[int(numbers[i])-1] = 1 #로또번호가 1부터 시작하지만 벡터의 인덱스 시작은 0부터 시작하므로 1을 뺌
    
    return ohbin

# 원핫인코딩벡터(ohbin)를 번호로 변환
def ohbin2numbers(ohbin):

    numbers = []
    
    for i in range(len(ohbin)):
        if ohbin[i] == 1.0: # 1.0으로 설정되어 있으면 해당 번호를 반환값에 추가한다.
            numbers.append(i+1)
    
    return numbers
     

In [5]:
rows = df.values
row_count = len(rows)

numbers = rows[:, 1:7]
lotto_onehot = list(map(numbers2ohbin, numbers))

x_samples = lotto_onehot[0:row_count-1]
y_samples = lotto_onehot[1:row_count]

#원핫인코딩으로 표시
print("lotto_onehot")
print("X[0]: " + str(x_samples[0]))
print("Y[0]: " + str(y_samples[0]))

#번호로 표시
print("numbers")
print("X[0]: " + str(ohbin2numbers(x_samples[0])))
print("Y[0]: " + str(ohbin2numbers(y_samples[0])))

lotto_onehot
X[0]: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
Y[0]: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
numbers
X[0]: [16, 23, 29, 33, 37, 40]
Y[0]: [2, 13, 21, 25, 32, 42]


In [14]:
ohbin2numbers(list(lotto_df.iloc[0]))

[16, 23, 29, 33, 37, 40]

In [6]:
lotto_df = pd.DataFrame(lotto_onehot, columns=np.arange(1,46), index = np.arange(1, 1087))
# lotto_df = lotto_df.astype(int)
lotto_df

,1,2,3,4,5,6,7,8,9,10,...,36,37,38,39,40,41,42,43,44,45
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1083,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1084,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1085,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [36]:
X = lotto_df.reset_index()[['index']]
y = lotto_df.values#[0].reshape(-1, 1)

print(x)
print(y)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Modeling

In [88]:
# Model Load
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
# xgb = xgb.XGBClassifier()
# lgb = lgb.LGBMClassifier()

# Training 
dt.fit(X, y)
rf.fit(X, y)
# xgb.fit(X_train, y_train)
# lgb.fit(X_train, y_train)

# Predict 
# dt_pred = dt.predict(X_val)
# rf_pred = rf.predict(X_val)
# xgb_pred = xgb.predict(X_val)
# lgb_pred = lgb.predict(X_val)

RandomForestClassifier()

In [91]:
# 실제값
print('Actual :', ohbin2numbers(list(lotto_df.loc[1086])))

# 예측값 
print('Predict :', ohbin2numbers(rf.predict([[1087]]).reshape(-1, 1)))

Actual : [16, 25, 27, 35, 36, 37]
Predict : [16, 25, 27, 35, 36, 37]


Evaluation

In [ ]:
# accuracy, recall, precision, auc graph, confusion matrix

def eval_model(pred_list, name_list, y_test):
    for pred, name in zip(pred_list, name_list):
        accuracy = accuracy_score(y_test , pred)
        recall = recall_score(y_test, pred)
        precision = precision_score(y_test, pred)
        auc = roc_auc_score(y_test, pred)
        matrix = confusion_matrix(y_test, pred)

        plt.figure(figsize=(8,3))
        plt.subplot(1, 2, 1)
        fpr, tpr, thresholds = roc_curve(y_val, pred, pos_label=1)
        # roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, color='darkorange')
        plt.plot([0, 1], [0, 1], color = 'black', label = 'y = x')
        plt.subplot(1, 2, 2)
        sns.heatmap(matrix, annot=True, fmt='g') # fmt='g' : 지수 표기를 일반 표기로
        plt.title('{0} Model Confusion Matrix'.format(name))
        plt.show()
        plt.tight_layout()

        print('{0} 정확도: {1:.4f}'.format(name, accuracy))
        print('{0} Recall: {1:.4f}'.format(name, recall))
        print('{0} Precision: {1:.4f}'.format(name, precision))
        print('{0} AUC: {1:.4f}'.format(name, auc))

        print('='* 50)
        print()